# Fix My Analysis: Quarterly Sales Report

You are a Data Analyst at an e-commerce company. The previous analyst created this report, but there are several errors that need to be fixed before it can be sent to management.

**Your Goal:** Work through the notebook, identify and fix the errors, and document your changes.

## Step 1: Load the Data

In [ ]:
import pandas as pd

# Load the datasets
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
orders = pd.read_csv("orders.csv")

print("Datasets loaded!")
orders.head()

## Step 2: Combine the Datasets

The first step is to combine these datasets into a single DataFrame for analysis.

In [ ]:
# Combine orders with customer and product data
merged_data = orders.merge(customers, on="customer_id", how="inner")
merged_data = merged_data.merge(products, on="product_id", how="inner")

print("Data merged!")
merged_data.info()

### **Task 1: Check the Join Strategy**

**Context:** The previous analyst used `inner` joins. Is this the best approach? An `inner` join will only keep records that exist in *both* datasets.

**Investigation:** Check if we lost any order records after the joins. The original `orders` DataFrame has 500 rows. Does `merged_data` also have 500 rows?

**Hint:** Think about what happens if an order has a `customer_id` or `product_id` that doesn't exist in the other tables. Should we use a `left` join instead to keep all orders?

In [ ]:
# Your code here to investigate the number of rows

## Step 3: Data Cleaning

Now let's clean the data. We'll calculate the total price for each order and convert the date column.

In [ ]:
# Calculate total price - FLAWED CALCULATION
merged_data["total_price"] = merged_data["quantity"] + merged_data["unit_cost"]

# Convert order_date to datetime
merged_data["order_date"] = pd.to_datetime(merged_data["order_date"])

print("Data cleaned!")
merged_data.describe()

### **Task 2: Fix the Total Price Calculation and Check for Issues**

**Context:** Look at the code above that calculates `total_price` and correct it if necessary

**Investigation:** 
1.  Identify the error in the `total_price` calculation and fix it.
2.  Look at the `describe()` output. The `min` value for `quantity` is negative. Find and examine orders with invalid quantities (negative or zero values).
3.  Check for orders with invalid `unit_cost` values (zero or negative).
4.  Remove all orders with invalid data (negative/zero quantity or negative/zero unit cost).
5.  Recalculate the `total_price` with the corrected formula after removing invalid data.



In [ ]:
# Your code here to fix the calculation and remove invalid data

### **Task 3: Check for Missing Data**

**Context:** Missing data can cause problems in analysis. We should check how much missing data we have across all columns.

**Investigation:**
1.  Count the number of missing values (NaN) in each column of the dataset.
2.  Calculate the percentage of missing values for each column.
3.  Identify which columns have missing data.
4.  Decide how to handle the missing data. (You have been told missing customer information is acceptable, but orders with missing product information should be removed)

**Hint:** Use `.isnull().sum()` to count missing values per column.

In [ ]:
# Your code here to check for and handle missing data

## Step 4: Aggregate the Data

Let's create a summary of sales by product category.

In [ ]:
# Aggregate sales by category
category_summary = merged_data.groupby("category").agg({
    "total_price": "mean",
    "order_id": "count"
}).reset_index()

category_summary.columns = ["Category", "Average Sale Value", "Number of Orders"]

print("Sales Summary by Category:")
category_summary

### **Task 4: Check the Aggregation Logic**

**Context:** The summary above shows the *average* sale value. Is this the best metric for a sales report? Management usually wants to see the *total* sales.

**Investigation:** Change the aggregation function to get the total sales per category. Does this give a better picture of performance? Also, sort the results by total sales in descending order.

In [ ]:
# Your code here to fix the aggregation

## Step 5: Visualise the Results

Let's visualise the category summary with a bar chart.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a bar chart 
plt.figure(figsize=(10, 6))
sns.barplot(data=category_summary, x="Category", y="Number of Orders")
plt.title("Total Sales Value by Category")
plt.xlabel("Category")
plt.ylabel("Total Sales Value (£)")
plt.show()

### **Task 5: Fix the Visualisation**

**Context:** Look carefully at the chart above. The title says "Total Sales Value by Category" but what is actually being plotted? Check the code to see what data is being visualised.

**Investigation:**
1.  Identify what's wrong with the visualisation (hint: look at the `y` parameter).
2.  Create a corrected bar chart using your fixed summary table from Task 4 (with total sales, not average).
3.  Make sure the chart plots the correct metric and has accurate labels.
4.  Sort the bars by sales value to make it easier to read.

**Hint:** The y-axis should show total sales value, not number of orders!

In [ ]:
# Your code here to create a corrected visualisation

## Step 6: Geographical Sales Analysis

Now let's analyse sales by customer region.

In [ ]:
# Geographical Sales Analysis - FLAWED: using wrong aggregation
region_summary = merged_data.groupby("region").agg({
    "total_price": "mean",
    "customer_id": "count"
}).reset_index()

region_summary.columns = ["Region", "Average Sale Value", "Number of Customers"]

print("Sales Summary by Region:")
region_summary

### **Task 6: Fix the Regional Analysis**

**Context:** The regional analysis has similar problems to the category analysis. We want to see total sales by region, not average sales.

**Investigation:**
1.  Change the aggregation 
2.  The column name says "Number of Customers" but we're counting `customer_id` which might count the same customer multiple times. Change it to provide the "Number of Orders".
3.  Sort the results by total sales in descending order.
4.  Create a bar chart showing total sales by region (sorted by sales value).

**Hint:** Use the same approach you used to fix the category analysis!

In [ ]:
# Your code here to fix the regional analysis

## Step 7: Monthly Sales Trend Analysis

Let's look at how the value of sales have changed over time.

In [ ]:
# Monthly Sales Trend Analysis 
merged_data["order_month"] = merged_data["order_date"].dt.to_period("M")
monthly_summary = merged_data.groupby("order_month").agg({
    "order_id": "count"
}).reset_index()

monthly_summary.columns = ["Month", "Total Sales"]
monthly_summary["Month"] = monthly_summary["Month"].dt.to_timestamp()

print("Monthly Sales Summary:")
monthly_summary.head()

In [ ]:
# Visualisation 
plt.figure(figsize=(14, 7))
sns.lineplot(data=monthly_summary, x="Month", y="Total Sales")
plt.title("Monthly Sales Trend")
plt.xlabel("Month")
plt.ylabel("Total Sales Value (£)")
plt.grid(True)
plt.show()

### **Task 7: Fix the Time-Series Analysis**

**Context:** The monthly trend analysis has a critical error. Look at the aggregation code - what metric is actually being calculated?

**Investigation:**
1.  Identify what's wrong with the monthly aggregation (hint: we're counting orders, not summing sales).
2.  Fix the aggregation to calculate the sum of `total_price` for each month.
3.  Update the column name to accurately reflect what's being measured.
4.  Recreate the line chart with the corrected data.

**Hint:** The chart should show total sales revenue per month, not the number of orders!

In [ ]:
# Your code here to fix the time-series analysis

## Step 8: Final Summary & Documentation

### **Task 8: Document Your Fixes**

Now that you have fixed the analysis, the final step is to document your work. In a real-world scenario, you would update the `README.md` file in your repository.

**For this exercise, write a brief summary in the markdown cell below explaining:**
1.  What problems you found in the original analysis.
2.  How you fixed each problem.
3.  What the impact of your fixes was on the final results.
4.  What insights you gained from the corrected geographical and time-series analyses.

*Your summary here...*